In [1]:
import pandas as pd

In [10]:
#Carico tabella e rimuove le colonne che non ci interessano
filename="TCGA.GBM.mutect.da904cd3-79d7-4ae3-b6c0-e7127998b3e6.DR-10.0.somatic.maf"
path="data/"+filename
columns=["Hugo_Symbol","Variant_Classification","Tumor_Sample_Barcode","t_depth","t_alt_count"]
frame=pd.read_csv(path,sep="\t")[columns]
#1condizione_elimino tutte le righe che hanno variant classification uguale a Silent
frame=frame[frame.Variant_Classification!="Silent"]
#ora non ci serve più
columns.remove("Variant_Classification")
frame=frame[columns]


   Hugo_Symbol Variant_Classification          Tumor_Sample_Barcode  t_depth  \
0       EIF4G3      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08      138   
1         RPL5        Frame_Shift_Del  TCGA-06-5413-01A-01D-1696-08       91   
2       SH2D1B      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08       88   
3         RYR2      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08       18   
5        AMER3      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08       95   
7          TTN      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08       53   
8        HDLBP      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08      113   
12        CD86      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08      147   
13      PIK3CA      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08       98   
14       PPEF2      Missense_Mutation  TCGA-06-5413-01A-01D-1696-08      101   

    t_alt_count  
0            49  
1            35  
2            36  
3             5  
5            34  
7          

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19,20,28,87,88,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
#sistemo il sampleid
def join(v,sep):
    t=""
    for s in v:
        t+=s+"-"
    return t[0:len(t)-1]
#definisco lambda function per ottenere 3 token iniziali
takeTokens= lambda x:   join(x.split("-")[0:3],"-")
column=frame["Tumor_Sample_Barcode"]
column=column.apply(takeTokens)
frame["Tumor_Sample_Barcode"]=column

print (frame[0:10])


   Hugo_Symbol Variant_Classification Tumor_Sample_Barcode  t_depth  \
0       EIF4G3      Missense_Mutation         TCGA-06-5413      138   
1         RPL5        Frame_Shift_Del         TCGA-06-5413       91   
2       SH2D1B      Missense_Mutation         TCGA-06-5413       88   
3         RYR2      Missense_Mutation         TCGA-06-5413       18   
5        AMER3      Missense_Mutation         TCGA-06-5413       95   
7          TTN      Missense_Mutation         TCGA-06-5413       53   
8        HDLBP      Missense_Mutation         TCGA-06-5413      113   
12        CD86      Missense_Mutation         TCGA-06-5413      147   
13      PIK3CA      Missense_Mutation         TCGA-06-5413       98   
14       PPEF2      Missense_Mutation         TCGA-06-5413      101   

    t_alt_count  
0            49  
1            35  
2            36  
3             5  
5            34  
7            22  
8            41  
12           39  
13           38  
14           41  


In [54]:
#calcolo la probabilità per il gene i e sample j facendo il min{1,2*t_alt_count/t_depth}
frame["prob"]=2*frame["t_alt_count"]/frame["t_depth"]
#print (len(frame[frame['prob']>1]))
frame[frame['prob']>1] = 1
print (frame[0:10])

5933
   Hugo_Symbol Variant_Classification Tumor_Sample_Barcode  t_depth  \
0       EIF4G3      Missense_Mutation         TCGA-06-5413      138   
1         RPL5        Frame_Shift_Del         TCGA-06-5413       91   
2       SH2D1B      Missense_Mutation         TCGA-06-5413       88   
3         RYR2      Missense_Mutation         TCGA-06-5413       18   
5        AMER3      Missense_Mutation         TCGA-06-5413       95   
7          TTN      Missense_Mutation         TCGA-06-5413       53   
8        HDLBP      Missense_Mutation         TCGA-06-5413      113   
12        CD86      Missense_Mutation         TCGA-06-5413      147   
13      PIK3CA      Missense_Mutation         TCGA-06-5413       98   
14       PPEF2      Missense_Mutation         TCGA-06-5413      101   

    t_alt_count      prob  
0            49  0.710145  
1            35  0.769231  
2            36  0.818182  
3             5  0.555556  
5            34  0.715789  
7            22  0.830189  
8            41  

In [80]:
#
group=frame.groupby(["Hugo_Symbol","Tumor_Sample_Barcode"])
#print(type(group))
#print(len(group))
#print(len(frame))
group=group.aggregate(max)
#print(type(group))


<class 'pandas.core.groupby.DataFrameGroupBy'>
53210
66224
<class 'pandas.core.frame.DataFrame'>


In [81]:
#riporto gli indici (geneid,sampleid) come colonne del frame
#ed eliminio prima riga che non so perche compaia
columns=["prob"]
columns2=["Hugo_Symbol","Tumor_Sample_Barcode"]
group=group[1:len(group)]
group.reset_index(level=0, inplace=True)
group.reset_index(level=0, inplace=True)

print(group[0:5])
print(group.columns.values)
print(group.index)

  Tumor_Sample_Barcode Hugo_Symbol Variant_Classification  t_depth  \
0         TCGA-06-5411        A1BG      Missense_Mutation       48   
1         TCGA-19-0957        A1BG        Frame_Shift_Ins       68   
2         TCGA-19-5956        A1BG                 Intron       28   
3         TCGA-14-0813        A1CF      Missense_Mutation      104   
4         TCGA-14-1794        A1CF      Missense_Mutation       82   

   t_alt_count      prob  
0           20  0.833333  
1            8  0.235294  
2            4  0.285714  
3           10  0.192308  
4            8  0.195122  
['Tumor_Sample_Barcode' 'Hugo_Symbol' 'Variant_Classification' 't_depth'
 't_alt_count' 'prob']
RangeIndex(start=0, stop=53209, step=1)


In [82]:
#seleziono solo alcune colonne e le rinomino
columns=["Hugo_Symbol","Tumor_Sample_Barcode","prob"]
columns_rename=["geneid","sampleid","prob"]
group=group[columns]
group.columns = columns_rename

In [83]:
print(group[0:5])

  geneid      sampleid      prob
0   A1BG  TCGA-06-5411  0.833333
1   A1BG  TCGA-19-0957  0.235294
2   A1BG  TCGA-19-5956  0.285714
3   A1CF  TCGA-14-0813  0.192308
4   A1CF  TCGA-14-1794  0.195122


In [85]:
#salvo tabella ottenuta
filename="matriceProb.csv"
group.to_csv(filename,sep="\t")

In [4]:
#carico tabella e creo dizionario dei samples/patients
filename="matriceProb.csv"
frame=pd.read_csv(filename,sep="\t")
print(frame[0:5])

   Unnamed: 0 geneid      sampleid      prob
0           0   A1BG  TCGA-06-5411  0.833333
1           1   A1BG  TCGA-19-0957  0.235294
2           2   A1BG  TCGA-19-5956  0.285714
3           3   A1CF  TCGA-14-0813  0.192308
4           4   A1CF  TCGA-14-1794  0.195122


In [7]:
patients={}
for index,row in frame.iterrows():
    patients.setdefault(row["sampleid"], {})
    patients[row["sampleid"]][row["geneid"]]=row["prob"]
   
    

{'TCGA-06-5411': {'A1BG': 0.8333333333333334, 'ABI2': 0.640625, 'ADGRA2': 0.6024096385542169, 'ANO9': 0.5416666666666666, 'ARHGEF28': 0.5783132530120482, 'ARL9': 0.4827586206896552, 'CARD6': 0.5771144278606966, 'CBR1': 0.4897959183673469, 'CFAP74': 0.5517241379310345, 'DNAH7': 0.2222222222222222, 'DSG4': 0.6114649681528662, 'GABRA6': 0.6666666666666666, 'HM13': 0.52, 'KIF16B': 0.24489795918367346, 'LOXL2': 0.13636363636363635, 'MASP2': 0.8947368421052632, 'NPAP1': 0.7619047619047619, 'NRIP1': 0.5555555555555556, 'OR4D11': 0.4303797468354431, 'OR6V1': 0.5, 'PCK1': 0.5882352941176471, 'PIGF': 0.52, 'PRSS58': 0.2777777777777778, 'PTEN': 0.5866666666666667, 'SPDYE3': 0.4, 'TLX3': 0.6666666666666666, 'TMEM107': 0.5161290322580645, 'TTLL11': 0.625}, 'TCGA-19-0957': {'A1BG': 0.2352941176470588, 'AASS': 0.10552763819095476, 'ABCC11': 0.20809248554913296, 'ABCG1': 0.5138888888888888, 'ACAD10': 0.5671641791044776, 'ACD': 0.631578947368421, 'ACPP': 0.07729468599033816, 'ACSBG2': 0.378378378378378